In [ ]:
import pandas as pd
import numpy as np
# import the stats packages i need
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
#import the training and validation data
train = pd.read_csv('../../0_DataPreparation/Trainingsdaten.csv')
#convert the ship colum from True and False to 1 and 0
train['Ship'] = train['Ship'].astype(int)
#add a colum with true/false for the day before a Feiertag
train['Feiertag-1'] = train['Feiertag'].shift(1)
print(train.head())
validierung = pd.read_csv('../../0_DataPreparation/Validierungsdaten.csv')
validierung['Ship'] = validierung['Ship'].astype(int)
validierung['Feiertag-1'] = validierung['Feiertag'].shift(1)

In [ ]:
#lets start with a simple linear regression model using the smf package
#we will use the training data
#fit the linear regression model
lm = smf.ols(formula='Ship ~ Umsatz_total', data=train).fit()
#print the summary of the model
print(lm.summary())

In [ ]:
#lets build a multiple linear regression model
#remove all rows wuth missing values from the training data
train = train.dropna()
#select the features for the model
features = ['Ship', 'Feiertag', 'Feiertag-1', 'Croissant', 'Konditorei']
x_train = train[features]
print(x_train.head())
#remove all rows wuth missing values
x_train = x_train.dropna()
y_train = train['Umsatz_total']

#lets scale the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

# Add a constant term to the training data for the regression intercept
x_train = sm.add_constant(x_train)

# Train the linear regression model using Statsmodels
model = sm.OLS(y_train, x_train).fit()

# Print the summary of the regression model
# Includes metrics like R-squared, coefficients, and p-values for each feature
print(model.summary())

#lets predict the validation data
validierung = validierung.dropna()
x_validierung = validierung[features]
print(x_validierung.head())
x_validierung = scaler.fit_transform(x_validierung)  # Use transform instead of fit_transform
x_validierung = sm.add_constant(x_validierung)  # Add constant after scaling
y_validierung = validierung['Umsatz_total']

#predict the validation data
predictions = model.predict(x_validierung)
print(predictions)

# Calculate Mean Absolute Percentage Error (MAPE) for error evaluation
mape = (abs((y_validierung - predictions) / y_validierung).mean()) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

